In [1]:
#Importing unityEnvironment work simulation to be done here along with numpy

from unityagents import UnityEnvironment
import numpy as np

In [2]:
#Loading Unity Environment with the downloaded environment app files

env = UnityEnvironment(file_name='../Reacher.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

brain_name, brain

('ReacherBrain', <unityagents.brain.BrainParameters at 0x11203bf98>)

In [4]:
env_info = env.reset(train_mode=True)[brain_name]
num_agents = len(env_info.agents)
action_size = brain.vector_action_space_size
num_states = len(env_info.vector_observations[0])

num_agents,action_size,num_states

(20, 4, 33)

In [5]:
from ddpg_agent import Agent

agent = Agent(num_states, action_size,0,num_agents)
agent

In [6]:
import random
import torch
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
update_num = 5

def ddpg(n_episodes=1000,max_t=500,toggle=False):
    
    #To keep track of the most recent scores for the environment
    scores_deque = deque(maxlen=100)
    
    #To keep track of all scores from the environment
    scores = []
    
    #Epsilon values
    epsilon = 1
    epsilon_end = 0.05
    epsilon_decay = 1e-4
    
    #Learning from num_episodes using experience replay
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        
        #To reset noise for the episode
        agent.reset()
        to_update=1
        score = np.zeros(num_agents)
        
        for t in range(max_t):
            
            #Will return an array of actions that according to the policy are optimal, using the actor network
            actions = agent.act(states,epsilon)
            
            #Taking one step for each agent using the above obtained array of actions 
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
            
            """
            Taking a step with the current set of states and actions which will be put to the 
            experience replay memory, and if the memory reaches a specific limit then the learning 
            will happen accordingly.
            """
            
            agent.step(states, actions, rewards, next_states, dones)
            
#             #Update networks every alternate step
#             if to_update:
#                 agent.update()
                
#             to_update = (to_update+1)%2

            for i in range(update_num):
                if to_update:
                    agent.update()
            
            if toggle:
                    to_update = (to_update+1)%2
                    
            #Preparing agent for next step
            states = next_states
            score += rewards
            
            #if any agent is done with their respective episode, then break
            if np.any(dones):
                break
        
        #Update epsilon
        epsilon = max(epsilon-epsilon_decay,epsilon_end)
        
        #Append mean of scores for all agents to the score arrays
        scores_deque.append(np.mean(score))
        scores.append(np.mean(score))
        
        #Print average score after each episode
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        
        if np.mean(scores_deque)>=30.:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.
                  format(i_episode, np.mean(scores_deque)))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor2.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic2.pth')
            break
    return scores
    

scores = ddpg()

#Plotting the graph with the mean scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

/Users/championballer/Documents/RL/P2_Continuous_Actions/Final submission/ddpg_agent.py:126: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self.critic_local.parameters(),1)


KeyboardInterrupt: 